## Define useful functions

In [53]:
import torchvision
import numpy as np
import torch 
import matplotlib.pyplot as plt
from glob import glob

def create_mask(patch,thres):
    patch_array=np.asarray(patch,dtype='uint8')
    avg_img=torch.div(torch.sum(torch.tensor(patch_array).permute(2, 0, 1),0,keepdim=True),3)
    tr=torchvision.transforms.GaussianBlur(7, sigma=5.0)
    img_tr=tr(avg_img)
    rough_mask=torch.where(img_tr<thres,1,0)
    return(rough_mask)

def plot_mask_img(patch, thres):
    figure, axs= plt.subplots(ncols=2, figsize=(15,15))
    axs[0].imshow(np.array(patch))
    axs[1].imshow(create_mask(patch, thres).squeeze(0))

def load_image(path):
    return(np.array(Image.open(path)))

## Load tiles

In [57]:
from PIL import Image

# Path to tiles
src_dict="E:\\02_TILES\\002_TILES CTRL\\10X\\"

# Get patients
patients = glob(src_dict+'*HES')
patients_interest=patients
patient_patch_interest=[glob(patient+'/*.tiff')[10] for patient in patients_interest]


## Open a tile and visualize it

In [63]:
patch=Image.open(patient_patch_interest[20])
plot_mask_img(patch, thres=210)

## Tile filtering

In [ ]:
import os

# threshold used to identify tissue
thres= 210
# minimum tissue proportion 
n= 30 #pourcentage de tissu minimum
# directory used to put all filtered out tiles
new_dir= 'E:\\02_TILES\\002_TILES CTRL\\5X_too_few'

# actual processing
for patient in patients:
    patch_list=glob(patient+'/*.tiff')
    count=0
    for patch in patch_list:
        img=load_image(patch)
        mask=create_mask(img,thres)
        if torch.sum(mask) < img.shape[1]**2*n/100: #Défini pour n% de tissu
            count+=1
            os.makedirs(new_dir+'/'+patient.split('\\')[-1],exist_ok=True)
            new_path=new_dir+'/'+patient.split('\\')[-1]+'/'+patch.split('\\')[-1]
            os.rename(patch,new_path)